In [2]:
#####Hierarchical bayes individual RFM model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import itertools
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [5]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = (upper - lower) *rand(col*col) + lower   #相関係数の乱数ベクトルを作成
    rho = np.reshape(np.array(cov_vec), (col, col)) * np.tri(col)   #乱数ベクトルを下三角行列化
    Sigma = np.diag(np.diag(rho + rho.T) + 1) - (rho + rho.T)   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = scipy.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(eigen_val.shape[0]-1):
        if eigen_val[i] < 0:
            eigen_val[i] = (eigen_upper - eigen_lower) * rand(1) + eigen_lower
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(pow(np.diag(Sigma), 0.5)[:, np.newaxis], pow(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

##
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [3]:
####データの発生####
##データの設定
k = 3
hh = 10000   #ユーザー数
period = 200   #観測期間

In [33]:
##階層モデルの説明変数を生成
#ユーザーの説明変数を生成
k1 = 3; k2 = 5; k3 = 5
u1 = np.array(np.random.random(hh*k1)).reshape((hh, k1))
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hh)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, hh)[:, np.newaxis], u1, u2, u3), axis=1)
u_col = u.shape[1]

In [45]:
##ユーザーごとにデータを生成
#分散共分散行列を設定
Cor = np.identity((k))
Cor[1, 0] = Cor[0, 1] = 0.5; Cor[2, 1] = Cor[1, 2] = 0.4; Cor[0, 2] = Cor[2, 0] = 0
tau = np.array([0.1, 0.15, 0.05])
Cov = covmatrix(Cor, np.sqrt(tau), np.sqrt(tau))
Covt = Cov.copy()

#階層モデルの回帰パラメータ
beta1 = np.array([1.75, 4.25, 1.5])
beta2 = np.hstack((np.random.uniform(-0.55, 0.75, (k-1)*(u_col-1)).reshape(u_col-1, k-1), 
                   np.random.uniform(-0.5, 0.5, u_col-1).reshape(u_col-1, 1)))
beta = np.vstack((beta1, beta2))
betat = beta.copy()

#階層モデルからモデルパラメータを生成
theta = np.dot(u, beta) + np.random.multivariate_normal(np.repeat(0, k), Cov, hh)
Sigma = 0.75
thetat = theta.copy(); Sigmat = Sigma



In [30]:
Cor

array([[1. , 0.5, 0. ],
       [0.5, 1. , 0.4],
       [0. , 0.4, 1. ]])